# **Skenario 1 - CNN VGG16**

Skenario 1 menggunakan model Convolutional Neural Network (CNN) dengan arsitektur VGG16 untuk melakukan klasifikasi pada data citra otak.

---

## 1. Import packages

Import modul-modul yang diperlukan untuk pengolahan citra medis, termasuk pembacaan citra dengan `SimpleITK`, manipulasi array dengan `NumPy`, visualisasi dengan `Matplotlib`, dan perhitungan entropy dengan `scipy.stats`

In [1]:
import os
import numpy as np
import SimpleITK as sitk
from PIL import Image
import tensorflow as tf
import cv2

## 2. Data Preparation

### 2.1 Load Data

cari dan cetak daftar file dengan ekstensi `.npy` di dalam struktur direktori yang dimulai dari base_dir.

In [9]:
base_dir = 'D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data'

def get_npy_files(base_dir):
    npy_files = []

    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.lower().endswith('.npy'):
                npy_files.append(os.path.join(root, file))

    return npy_files

npy_files = get_npy_files(base_dir)

print("Daftar File dengan Ekstensi .npy:")
for file in npy_files:
    print(file)

Daftar File dengan Ekstensi .npy:
D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_1\Testing\slice1.npy
D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_1\Testing\slice10.npy
D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_1\Testing\slice11.npy
D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_1\Testing\slice12.npy
D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_1\Testing\slice13.npy
D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_1\Testing\slice14.npy
D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_1\Testing\slice15.npy
D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_1\Testing\slice16.npy
D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_1\Testing\slice17.npy
D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIME

hitung dan cetak jumlah file gambar dalam format `.npy` yang terletak di dalam subfolder `training` dan `testing` untuk setiap label dan fold yang ada dalam struktur direktori yang dimulai dari base_dir.

In [11]:
def load_images_and_count_sitk(base_dir):
    total_training, total_testing = 0, 0

    for label_folder in [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]:
        label_folder_path = os.path.join(base_dir, label_folder)
        
        fold_folders = [f for f in os.listdir(label_folder_path) if f.startswith('Fold_') and os.path.isdir(os.path.join(label_folder_path, f))]

        for fold_folder in fold_folders:
            fold_number = int(fold_folder.split('_')[1])
            training_folder_path = os.path.join(label_folder_path, fold_folder, 'training')
            testing_folder_path = os.path.join(label_folder_path, fold_folder, 'testing')

            if not all(map(os.path.exists, [training_folder_path, testing_folder_path])):
                print(f"Warning: Training or testing folder not found - {training_folder_path} or {testing_folder_path}")
                continue

            training_images = [f for f in os.listdir(training_folder_path) if f.lower().endswith('.npy')]
            testing_images = [f for f in os.listdir(testing_folder_path) if f.lower().endswith('.npy')]

            total_training += len(training_images)
            total_testing += len(testing_images)

            print(f"Label: {label_folder}, Fold: {fold_number}")
            print(f"  - Total Training Images: {len(training_images)}")
            print(f"  - Total Testing Images: {len(testing_images)}")

    print(f"Total Training Images Across Folds: {total_training}")
    print(f"Total Testing Images Across Folds: {total_testing}")

# Panggil fungsi dengan base_dir yang baru
load_images_and_count_sitk(base_dir)

Label: AD, Fold: 1
  - Total Training Images: 72
  - Total Testing Images: 18
Label: AD, Fold: 2
  - Total Training Images: 72
  - Total Testing Images: 18
Label: AD, Fold: 3
  - Total Training Images: 72
  - Total Testing Images: 18
Label: AD, Fold: 4
  - Total Training Images: 72
  - Total Testing Images: 18
Label: AD, Fold: 5
  - Total Training Images: 72
  - Total Testing Images: 18
Label: CN, Fold: 1
  - Total Training Images: 72
  - Total Testing Images: 18
Label: CN, Fold: 2
  - Total Training Images: 72
  - Total Testing Images: 18
Label: CN, Fold: 3
  - Total Training Images: 72
  - Total Testing Images: 18
Label: CN, Fold: 4
  - Total Training Images: 72
  - Total Testing Images: 18
Label: CN, Fold: 5
  - Total Training Images: 72
  - Total Testing Images: 18
Label: EMCI, Fold: 1
  - Total Training Images: 72
  - Total Testing Images: 18
Label: EMCI, Fold: 2
  - Total Training Images: 72
  - Total Testing Images: 18
Label: EMCI, Fold: 3
  - Total Training Images: 72
  - Total

In [22]:
print(type(load_images_and_count_sitk(base_dir)))

Label: AD, Fold: 1
  - Total Training Images: 72
  - Total Testing Images: 18
Label: AD, Fold: 2
  - Total Training Images: 72
  - Total Testing Images: 18
Label: AD, Fold: 3
  - Total Training Images: 72
  - Total Testing Images: 18
Label: AD, Fold: 4
  - Total Training Images: 72
  - Total Testing Images: 18
Label: AD, Fold: 5
  - Total Training Images: 72
  - Total Testing Images: 18
Label: CN, Fold: 1
  - Total Training Images: 72
  - Total Testing Images: 18
Label: CN, Fold: 2
  - Total Training Images: 72
  - Total Testing Images: 18
Label: CN, Fold: 3
  - Total Training Images: 72
  - Total Testing Images: 18
Label: CN, Fold: 4
  - Total Training Images: 72
  - Total Testing Images: 18
Label: CN, Fold: 5
  - Total Training Images: 72
  - Total Testing Images: 18
Label: EMCI, Fold: 1
  - Total Training Images: 72
  - Total Testing Images: 18
Label: EMCI, Fold: 2
  - Total Training Images: 72
  - Total Testing Images: 18
Label: EMCI, Fold: 3
  - Total Training Images: 72
  - Total

## 3. Data Processing

### 3.1 Resize Citra

In [12]:
resized_data_path = 'D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/ResizedData'

if not os.path.exists(resized_data_path):
    os.makedirs(resized_data_path)

In [24]:
target_size = (224, 224)

# Looping melalui label dan fold
for label_folder in os.listdir(base_dir):
    label_folder_path = os.path.join(base_dir, label_folder)

    # Pastikan itu adalah direktori label
    if os.path.isdir(label_folder_path):
        for fold_folder in os.listdir(label_folder_path):
            fold_folder_path = os.path.join(label_folder_path, fold_folder)

            # Pastikan itu adalah direktori fold
            if os.path.isdir(fold_folder_path) and fold_folder.startswith('Fold_'):
                train_path = os.path.join(fold_folder_path, 'Training')
                resized_train_path = os.path.join(fold_folder_path, 'ResizedTraining')

                # Membuat direktori untuk citra yang sudah diresize jika belum ada
                os.makedirs(resized_train_path, exist_ok=True)

                # Looping melalui setiap file citra di direktori data pelatihan
                for filename in os.listdir(train_path):
                    img_path = os.path.join(train_path, filename)

                    try:
                        # Membaca data citra dari file .npy
                        img_data = np.load(img_path, allow_pickle=True).astype('float32')

                        # Melakukan resize pada data citra
                        resized_img_data = cv2.resize(img_data, target_size)

                        # Menyimpan citra yang sudah diresize dalam format .npy
                        resized_filename = f'resized_{filename}'
                        resized_img_path = os.path.join(resized_train_path, resized_filename)
                        np.save(resized_img_path, resized_img_data)
                    except Exception as e:
                        print(f"Error processing {img_path}: {e}")

print("Proses resize data training selesai.")

Error processing D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_1\Training\slice1.npy: setting an array element with a sequence.
Error processing D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_1\Training\slice10.npy: setting an array element with a sequence.
Error processing D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_1\Training\slice11.npy: setting an array element with a sequence.
Error processing D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_1\Training\slice12.npy: setting an array element with a sequence.
Error processing D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_1\Training\slice13.npy: setting an array element with a sequence.
Error processing D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_1\Training\slice14.npy: setting an array element with a sequence.
Error processing D:/Doc

Error processing D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_4\Training\slice44.npy: setting an array element with a sequence.
Error processing D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_4\Training\slice45.npy: setting an array element with a sequence.
Error processing D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_4\Training\slice46.npy: setting an array element with a sequence.
Error processing D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_4\Training\slice47.npy: setting an array element with a sequence.
Error processing D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_4\Training\slice48.npy: setting an array element with a sequence.
Error processing D:/Documents/Kuliah/.SKRIPSI/KLASIFIKASI ALZHEIMER/Coding/Notebook/Data\AD\Fold_4\Training\slice49.npy: setting an array element with a sequence.
Error processing D:/Do

In [25]:
print(f"Sebelum resize - Dimensi citra {img_data.shape}")

Sebelum resize - Dimensi citra (2,)


## 4. Modeling

## 5. Training Model

## 6. Testing

## 7. Evaluation